# 1. Introduction to Python at WRDS

WRDS provides an interface for Python access directly, allowing querying of WRDS data within your Python program. 

This short tutorial follows closely from https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/

## 1.1 Installing WRDS Python Module

Assume you've installed Python, the next step is to install the Python wrds module:
- For Mac OSX: open Terminal.
- For Windows: launch the Anaconda Prompt app.
Run the following command:

<div class="alert alert-block alert-success">
pip install wrds
</div>

This command will download and install the <b>wrds</b> Python module.
The wrds module is open source. Check out <a href="https://github.com/wharton/wrds">GitHub Repository</a>.

## 1.1 The pgpass File

The pgpass file includes your WRDS username and password so that you do not need to enter them each time you wish to connect to WRDS within Python.

First, start python. Then enter:

In [ ]:
import wrds
db = wrds.Connection(wrds_username='your_username')
db.create_pgpass_file()

Where <code>wrds_username</code> is your own WRDS username (the same as your login to the WRDS website). You will be prompted once for your WRDS username and password on your first login, at the <code>Connection()</code> step, but then, after running <code>create_pgpass_file()</code> once, you should be able to connect from then on without needing to do so. Test this by disconnecting and reconnecting, using the following:

In [ ]:
db.close()
db = wrds.Connection(wrds_username='your_username')

You should be connected directly without needing to authenticate. If your WRDS username and your local computer username (that you're running Python as) happen to be the same (in this case, both <em>your_username</em> ), you may omit the <code>wrds_username</code> argument to <code>Connection()</code>.

# 2. Querying WRDS Data using Python

All Python programs intending to access WRDS data must include the following two lines at the beginning:

In [1]:
import wrds
db = wrds.Connection()

Loading library list...
Done


<p>The first line imports the Python <strong>wrds</strong> module, and the second initializes a new connection to the WRDS database backend and saves that connection as <code>db</code>.</p>
<p>This connection ideally requires that you have set up your <code>.pgpass</code> file as covered previously. Otherwise you will be prompted for your WRDS username and password each time you connect to WRDS in this fashion.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> Class accounts and IPAuth / Daypass accounts are not permitted to access WRDS in this manner and will receive an error if trying this connection. You must have your own, dedicated WRDS account in order to access WRDS from PYTHON.
</div>

## 2.1 Methods in the wrds Module
 

<p>The <strong>wrds</strong> module supports several methods of accessing WRDS data. The best way to learn about them is using the inline documentation available within a Python console. You can see what functions are available by entering <code>db</code> and then pressing Tab key to expand the list, as follows:</p>

In [ ]:
db.close()
db.connection()
db.list_libraries()
db.list_tables()
db.describe_table()
db.get_table()
db.raw_sql()
db.get_row_count()

<div class="alert alert-block alert-info">
<b>NOTE:</b> There are a few other methods beyond the above that you'll see when looking at the contextual help list.
</div>

<p>For an online description of each, use the <code>help()</code> function as follows:</p>

In [ ]:
help(db.get_table)
help(db.raw_sql)

## 2.2 Pandas and Numpy

<p>Your data results are always returned as a Pandas DataFrame<em>.</em> Pandas is a popular Python module that, together with NumPy, forms the basis for most numeric data manipulation in Python. The pandas module provides powerful data manipulation capabilities at a granular degree of control over your output.</p>
<p>In all examples provided in this document, the returned results variable <strong>data</strong> is a Pandas DataFrame.</p>

## 2.3 Limiting the Number of Records Returned

When working with large data sources, it is important to begin your research with small subsets of the data you eventually want to query. Limiting the number of returned records (also called observations) is essential while developing your code, as queries that involve large date ranges or query a large number of variables (column fields) could take a long time and generate large output files.

<b>IMPORTANT</b>: This is especially important if you are running Python locally from your computer, as the returned query output data is downloaded from WRDS to your computer. Even if you have a fast computer, a slow or intermittent Internet connection could cripple your research if you don't perform your queries carefully.

There are two ways of limiting the number of records (to say 5), depending on which method you use to get data:
<ul><li><code>get_table()</code> - for this method, you would use <code>obs=5</code></li>
    <li><code>raw_sql()</code> - for this method, you would use <code>LIMIT 5</code></li></ul>

The next two cells show an example of each:

In [3]:
db.get_table('djones', 'djdaily', columns=['date', 'dji'], obs=5)

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60


In [2]:
db.raw_sql('select date,dji from djones.djdaily LIMIT 5;', date_cols=['date'])

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60


Each of these will be explored in more detail below.

## 2.4 Querying the Dataset Structure (Metadata)

<p>Data at WRDS is organized in a hierarchical manner by vendor (e.g. <strong>crsp</strong>), referred to at the top-level as <em>libraries</em>. Each library contains a number of component tables or <em>datasets</em> (e.g. <strong>dsf</strong>) which contain the actual data in tabular format, with column headers called <em>variables</em> (such as <em>date</em>, <em>askhi</em>, <em>bidlo</em>, etc).</p>

<p>You can analyze the structure of the data through its metadata using the <strong>wrds</strong> module, as outlined in the following steps:</p>

<ol><li>List all available <em>libraries</em> at WRDS using <code>list_libraries()</code></li>
    <li>Select a library to work with, and list all available <em>datasets</em> within that library using <code>list_tables()</code></li>
    <li>Select a dataset, and list all available <em>variables</em> (column headers) within that dataset using <code>describe_table()</code></li></ol>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> When referencing library and dataset names, you must use all <em>lowercase</em>.
</div>

<p>Alternatively, a comprehensive list of all WRDS libraries is available at the <a href="https://wrds-www.wharton.upenn.edu/pages/support/data-wrds/dataset-list/">Dataset List</a>. This resource provides a listing of each library, their component datasets and variables, as well as a tabular database preview feature, and is helpful in establishing the structure of the data you're looking for in an easy manner from a Web browser.</p>

<p>1. Determine the <em>libraries</em> available at WRDS:</p>

In [4]:
db.list_libraries();

This will list all libraries available at WRDS. Though all libraries will be shown, you must have a valid, current subscription for a library in order to access it via Python, just as with SAS or any other supported programming language at WRDS. You will receive an error message indicating this if you attempt to query a table to which your institution does not have access.

<p>2. To determine the <em>datasets</em> within a given library:</p>

In [ ]:
db.list_tables(library="library")

<p>Where 'library' is a dataset, such as <strong>crsp</strong> or <strong>comp,</strong> as returned from step 1 above.</p> 

For example, to determine <em>datasets</em> within the <b>MSCI ESG KLD STATS</b> library with the vendor name <b>kld</b>:

In [5]:
db.list_tables(library="kld")

['history', 'kldnames']

<p>3. To determine the column headers (<em>variables</em>) within a given dataset:</p>

In [ ]:
db.describe_table(library="library", table="table")

<p>Where 'library' is a dataset such as <strong>crsp</strong> as returned from #1 above and 'table' is a component database within that library, such as <strong>msf</strong>, as returned from query #2 above. Remember that both the library and the dataset are case-sensitive, and must be all-lowercase.</p>

<p>Alternatively, a comprehensive list of all WRDS libraries is available via the <a href="https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm">WRDS Dataset List</a>. This online resource provides a listing of each library, their component datasets and variables, as well as a tabular database preview feature.</p>

<p>By examining the metadata available to us -- the structure of the data -- we've determined how to reference the data we're researching, and what variables are available within that data. We can now perform our actual research, creating <em>data queries</em>, which are explored in depth in the next section.</p>

## 2.5 Querying WRDS Data

<div class="rich-text"><p>The <strong>wrds</strong> module provides several methods that are useful in gathering data:</p><ul>
    <li><code>get_table()</code> - fetches data by matching library and dataset, with the ability to filter using different parameters. This is the easiest method of accessing data.</li>
    <li><code>raw_sql()</code> - executes a SQL query against the specified library and dataset, allowing for highly-granular data queries.</li><li><code>get_row_count()</code> - returns the number of rows in a given dataset.</li>
    </ul>
    <p>Each of these is discussed below.</p></div>

### 2.5.1 Using get_table() 

<div class="rich-text"><p>The method <code>get_table()</code> accepts the following parameters:</p>
    <ul><li><strong>library</strong> - the library to query</li>
        <li><strong>table</strong> - the dataset to query</li>
        <li><strong>columns</strong> - the columns (variables) to include in the query (optional)</li>
        <li><strong>obs</strong> - the number of observations (rows) to return (optional)</li>
        <li><strong>offset</strong> - the starting point of for the query (optional)</li></ul>
    <p>For additional parameters, and further explanation of each, use the built-in help: <code>help(db.get_table)</code></p><p>Here's an example that returns the first 5 rows of only the <strong>date</strong> and <strong>dji</strong> columns from the Dow Jones Index:</p></div>

In [5]:
data = db.get_table(library='djones', table='djdaily', columns=['date', 'dji'], obs=5)
data

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60


<div class="alert alert-block alert-warning">
<b>NOTE:</b> The <strong>library</strong> and <strong>table</strong> parameters are required, and are also positional. Therefore, should you chose, you may omit the label for these two parameters as long as you supply them in order.
</div>

The following example illustrates this:

In [6]:
data = db.get_table('djones', 'djdaily', columns=['date', 'dji'], obs=5)
data

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60


### 2.5.2 Using raw_sql()

<div class="rich-text"><p>The method <code>raw_sql()</code> accepts the following parameters:</p>
    <ul><li><strong>sql</strong> - the SQL string to query</li>
        <li><strong>date_cols</strong> - a list or dict of column names to parse as date (optional)</li>
        <li><strong>index_col</strong> - a string or list of column(s) to set as index(es) (optional)</li></ul>
    <p>For additional parameters, and further explanation of each, use the built-in help: <code>help(db.raw_sql)</code></p><p>Here's an example that does the same thing as <code>get_table()</code> example above, but uses SQL to select the data instead:</p></div>

In [7]:
data = db.raw_sql('select date,dji from djones.djdaily LIMIT 5;', date_cols=['date'])
data

,date,dji
0,1896-05-26,40.94
1,1896-05-27,40.58
2,1896-05-28,40.20
3,1896-05-29,40.63
4,1896-06-01,40.60


<div class="rich-text"><p>The <code>raw_sql()</code> method is by far the most useful and popular of the <strong>wrds</strong> module's methods, allowing powerful and granular control over your data processing. Writing SQL for use with this method of fairly straightforward. All data queries are constructed in SQL the following generic manner:</p>
    <p><code>select columns from library.dataset where column1 = value</code></p>
    <p>Notice the <em>dot notation</em> for the library and dataset. Unlike the other <strong>wrds</strong> methods, where <em>library</em> and <em>table</em> are specified separately, SQL queries instead use the two together to identify the data location. So, for example, a data query for the <em>dataset</em> <strong>msf</strong> within the <em>library</em> <strong>crsp</strong> would use the syntax <strong>crsp.msf</strong>, and the same goes for <strong>djones.djdaily</strong>.</p>
    <p>You'll likely be doing most of your work using the <code>raw_sql()</code> method.</p></div>

### 2.5.3 Using get_row_count()

<div class="rich-text"><p>The method <code>get_row_count()</code> only accepts the following two parameters:</p>
    <ul><li><strong>library</strong> - the library to query</li>
        <li><strong>table</strong> - the dataset to query</li></ul>
    <p>For additional information, use the built-in help: <strong>help(db.get_row_count)</strong></p><p>Here is an example that returns the row count for the Dow Jones Index:</p></div>

In [9]:
data = db.get_row_count('djones', 'djdaily')
data

28038

## 2.6 Joining Data from Separate Datasets

<div class="rich-text">
    <p>Data from separate datasets can be joined and analyzed together. The following example will join the Compustat Fundamentals data set (<strong>comp.funda</strong>) with Compustat's pricing dataset (<strong>comp.secm</strong>), and then query for total assets and liabilities mixed with monthly close price and shares outstanding.</p><p></p><p><strong>To join and query two Compustat datasets:</strong></p><p></p>
</div>

In [11]:
db.raw_sql("select a.gvkey, a.datadate, a.tic, a.conm, a.at, a.lt, b.prccm, b.cshoq "
           "from comp.funda a join comp.secm b "
           "on a.gvkey = b.gvkey and a.datadate = b.datadate "
           "where a.tic = 'IBM' and a.datafmt = 'STD' and a.consol = 'C' and a.indfmt = 'INDL' "
           "LIMIT 10")

,gvkey,datadate,tic,conm,at,lt,prccm,cshoq
0,006066,1962-12-31,IBM,INTL BUSINESS MACHINES CORP,2112.301,731.700,389.999567,NaN
1,006066,1963-12-31,IBM,INTL BUSINESS MACHINES CORP,2373.857,782.119,506.999353,NaN
2,006066,1964-12-31,IBM,INTL BUSINESS MACHINES CORP,3309.152,1055.072,409.499496,NaN
3,006066,1965-12-31,IBM,INTL BUSINESS MACHINES CORP,3744.917,1166.771,498.999146,NaN
4,006066,1966-12-31,IBM,INTL BUSINESS MACHINES CORP,4660.777,1338.149,371.499662,NaN
5,006066,1967-12-31,IBM,INTL BUSINESS MACHINES CORP,5598.668,1767.067,626.999512,NaN
6,006066,1968-12-31,IBM,INTL BUSINESS MACHINES CORP,6743.430,2174.291,314.999847,NaN
7,006066,1969-12-31,IBM,INTL BUSINESS MACHINES CORP,7389.957,2112.967,364.499359,113.717
8,006066,1970-12-31,IBM,INTL BUSINESS MACHINES CORP,8539.047,2591.909,317.749634,114.587
9,006066,1971-12-31,IBM,INTL BUSINESS MACHINES CORP,9576.219,2933.837,336.499634,115.534


<p>The code joins both datasets using a common <strong>gvkey</strong> identifier and date, querying IBM with a frequency of one year, resulting in a result of 55 observations (as of 2017). Running joined queries between large datasets can require large amounts of memory and execution time. It is recommended you limit the scope of your queries to reasonable sizes when performing joins.</p>

<div class="alert alert-block alert-warning">
<b>NOTE:</b> For this example, you would need an active subscription to both datasets.
</div>

## 2.7 Managing your Connections

<div class="rich-text">
    <p>WRDS users are permitted up to 5 simultaneous connections to our Postgres data backend. For Python users, that means you may use <code>wrds.Connection()</code> up to five times before being denied additional connections.</p>
    <p>The best way to manage this, is to properly close out your connection to WRDS once you are done with it. With the Python <strong>wrds</strong> module, you simply use the <code>close()</code> method like so:</p>
</div>

In [ ]:
import wrds
db = wrds.Connection()
data = db.raw_sql("select * from djones.djdaily")
db.close()

<div class="rich-text">
    <p>Using the above, we've connected to WRDS, downloaded the data we need, and disconnected. We can continue to use the results from our query (<strong>data</strong>) even after disconnecting and the previous connection now longer counts against us when we go to connect again.<br><br>You should always disconnect using <code>close()</code> when you:</p>
    <ul><li>Exit your Python environment</li><li>Finish running your program</li>
        <li>Complete your data query download step, and want to move onto another.</li>
    </ul>
    <p>WRDS is currently working on a facility to allow you to cancel any connections you may have accidentally left open, but in the meantime, if you are getting an error message indicate that you have too many connections open to WRDS, try to close any existing connection you may have with the <code>close()</code> method, wait for your existing connections to timeout and close on their own, or reach out to us at <a href="https://wrds-web.wharton.upenn.edu/wrds/zendesk_sso.cfm">WRDS Support</a> for assistance.</p>
</div>

## 2.8 Passing Parameters to SQL 

<p>The <code>raw_sql()</code> method now also supports parameterized SQL, allowing you to pass variables or lists from elsewhere in your Python code to your SQL statement. This is great for large lists of company codes or identifiers, or an array of specific trading days. Here is an example where a dictionary of tickers is passed through to a raw_sql() SQL statement:</p>

In [ ]:
parm = {'tickers': ('0015B', '0030B', '0032A', '0033A', '0038A')}
data = db.raw_sql('SELECT datadate,gvkey,cusip FROM comp.funda WHERE tic in %(tickers)s', params=parm)

<div class="rich-text">
    <p>This allows for a great deal of flexibility in terms of your SQL queries. Common use cases might include building out a list of tickers, CUSIPS, etc programmatically or from an external file; re-using the same code list over multiple queries that adjust other parameters, such as date range; or matching based on specified trading days.</p>
    <p>This functionality uses Panda's native SQL capabilities, and thus requires SQL that conforms to a slightly different standard than the generalized SQL you otherwise use with <code>raw_sql()</code>. Specifically, it uses the Python DB-API 2.0 specification, which conforms to PEP 249.</p><p>For more information, please se one of the following links:</p>
    <p><a href="http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries">Psycopg: Passing Parameters to SQL Queries</a></p>
    <p><a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_query.html">Pandas: read_sql_query()</a></p><p><a href="https://www.python.org/dev/peps/pep-0249/">Python: PEP 249</a></p>
    <p>This feature was added in version 3.0.6 of the Python <strong>wrds</strong> module.</p>
</div>